## Webscrapping

O intuito desse notebook é realizar a coleta das informações: título, nota, duração, ano de lançamento, direção e principais artistas do Top 250 filmes do site IMDb.

Para esse processo serão importadas as bibliotecas:

In [1]:
import pandas as pd #para trabalhar com os dados extraídos em forma de DataFrame
import requests #para requisitar páginas
from bs4 import BeautifulSoup #quebrando o código para torná-lo acessível por suas tags

Após entender como as informações estão distribuídas, o primeiro passo é coletar o link referente a cada um dos filmes. Estes links estão disponíveis na página que contém a lista  de filmes.

In [37]:
url = 'https://www.imdb.com/chart/top/?ref_=nv_mv_250'

#Organizando informações do link com a lista de filmes
r = requests.get(url)
text = r.text
soup = BeautifulSoup(text) 

#Coletando links na href de cada filme
l = soup.find_all('tr')[1:] #extraindo todas as tag tr e criando uma lista

#extraindo apenas a referência do link
titles = []
for i in l:
    titles.append(i.find('a')['href']) 
    
links = []
#criando link de cada filme
for i in titles:
    links.append('https://www.imdb.com'+i)

links[:5]

['https://www.imdb.com/title/tt0111161/',
 'https://www.imdb.com/title/tt0068646/',
 'https://www.imdb.com/title/tt0468569/',
 'https://www.imdb.com/title/tt0071562/',
 'https://www.imdb.com/title/tt0050083/']

Com os links coletados é possível obter as informações de cada filme, individualmente.

In [76]:
#Coletando o código da página de cada filme

url = links[0]

r = requests.get(url)
text = r.text
soup = BeautifulSoup(text) 

In [112]:
#Coletando nome

nome = soup.find('h1').text

In [113]:
#Coletando nota
nota = soup.find('span', {'class':'sc-7ab21ed2-1 jGRxWM'}).text

In [114]:
#Coletando ano de lançamento
ano = soup.find('a', {'class':'ipc-link ipc-link--baseAlt ipc-link--inherit-color sc-8c396aa2-1 WIUyh'}).text

In [116]:
#Coletando diretor
diretor = soup.find('a', {'class':'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'}).text

In [117]:
#Coletando elenco
a = []
ator = soup.find_all('div', {'class':'ipc-metadata-list-item__content-container'})[2]
for i in ator.find_all('a'):
    a.append(i.text)

In [118]:
#Coletando duração
tempo = soup.find_all('li', {'class':'ipc-inline-list__item'})
tempo = tempo[2].text

##### Unificando código e iterando

In [109]:
d = {}
nome = []
nota = []
diretor = []
ano = []
duracao = []
elenco = []

for l in links:
    
    url = l

    r = requests.get(url)
    text = r.text
    soup = BeautifulSoup(text) 
    
    nome.append(soup.find('h1').text)
    nota.append(soup.find('span', {'class':'sc-7ab21ed2-1 jGRxWM'}).text)
    diretor.append(soup.find('a', 
                            {'class':'ipc-metadata-list-item__list-content-item ipc-metadata-list-item__list-content-item--link'}).text)
    ano.append(soup.find('a', 
                         {'class':'ipc-link ipc-link--baseAlt ipc-link--inherit-color sc-8c396aa2-1 WIUyh'}).text)
    tempo = soup.find_all('li', {'class':'ipc-inline-list__item'})
    duracao.append(tempo[2].text)
    a = []
    ator = soup.find_all('div', {'class':'ipc-metadata-list-item__content-container'})[2]
    for i in ator.find_all('a'):
        a.append(i.text)
    elenco.append(a)
    d['nome']=nome
    d['nota']=nota
    d['diretor']=diretor
    d['ano']=ano
    d['duracao']=duracao
    d['elenco']=elenco

In [107]:
df = pd.DataFrame.from_dict(d)

In [110]:
df

,nome,nota,diretor,ano,duracao,elenco
0,Um Sonho de Liberdade,9.3,Frank Darabont,1994,2h 22m,"[Tim Robbins, Morgan Freeman, Bob Gunton]"
1,O Poderoso Chefão,9.2,Francis Ford Coppola,1972,2h 55m,"[Marlon Brando, Al Pacino, James Caan]"
2,Batman: O Cavaleiro das Trevas,9.0,Christopher Nolan,2008,2h 32m,"[Christian Bale, Heath Ledger, Aaron Eckhart]"
3,O Poderoso Chefão II,9.0,Francis Ford Coppola,1974,3h 22m,"[Al Pacino, Robert De Niro, Robert Duvall]"
4,12 Homens e uma Sentença,9.0,Sidney Lumet,1957,1h 36m,"[Henry Fonda, Lee J. Cobb, Martin Balsam]"
...,...,...,...,...,...,...
245,Aladdin,8.0,Ron Clements,1992,1h 30m,"[Scott Weinger, Robin Williams, Linda Larkin]"
246,Histórias Cruzadas,8.1,Tate Taylor,2011,2h 26m,"[Emma Stone, Viola Davis, Octavia Spencer]"
247,A Bela e a Fera,8.0,Gary Trousdale,1991,1h 24m,"[Paige O'Hara, Robby Benson, Jesse Corti]"
248,Dança com Lobos,8.0,Kevin Costner,1990,3h 1m,"[Kevin Costner, Mary McDonnell, Graham Greene]"


In [111]:
df.to_csv('imdb_250.csv')